<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


- Install the additional package requirements for this bonus notebook by uncommenting and running the following cell:

In [1]:
# pip install -r requirements-extra.txt

# Comparing Various Byte Pair Encoding (BPE) Implementations

<br>
&nbsp;

## Using BPE from `tiktoken`

In [2]:
from importlib.metadata import version

print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.7.0


In [3]:
import tiktoken

tik_tokenizer = tiktoken.get_encoding("gpt2")

text = "Hello, world. Is this-- a test?"

In [4]:
integers = tik_tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


In [5]:
strings = tik_tokenizer.decode(integers)

print(strings)

Hello, world. Is this-- a test?


In [6]:
print(tik_tokenizer.n_vocab)

50257


<br>
&nbsp;

## Using the original BPE implementation used in GPT-2

In [7]:
from bpe_openai_gpt2 import get_encoder, download_vocab

In [8]:
download_vocab()

Fetching encoder.json: 1.04Mit [00:04, 249kit/s]                                                    
Fetching vocab.bpe: 457kit [00:01, 318kit/s]                                                        


In [14]:
orig_tokenizer = get_encoder(model_name="gpt2_model", models_dir=".")

In [15]:
integers = orig_tokenizer.encode(text)

print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


In [16]:
strings = orig_tokenizer.decode(integers)

print(strings)

Hello, world. Is this-- a test?


<br>
&nbsp;

## Using the BPE via Hugging Face transformers

In [19]:
import transformers

transformers.__version__

'4.42.4'

In [20]:
from transformers import GPT2Tokenizer

hf_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [21]:
hf_tokenizer(strings)["input_ids"]

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]

<br>
&nbsp;

## A quick performance benchmark

In [28]:
with open('../01_main-chapter-code/the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()

In [29]:
%timeit orig_tokenizer.encode(raw_text)

19.1 ms ± 626 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
%timeit tik_tokenizer.encode(raw_text)

5.29 ms ± 497 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [33]:
%timeit hf_tokenizer(raw_text)["input_ids"]

36.2 ms ± 1.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
len(hf_tokenizer(raw_text)["input_ids"])

5145

In [35]:
%timeit hf_tokenizer(raw_text, max_length=5145, truncation=True)["input_ids"]

36.9 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
len(hf_tokenizer(raw_text)["input_ids"])

5145

In [37]:
%timeit hf_tokenizer(raw_text, max_length=1024, truncation=True)["input_ids"]

34.5 ms ± 849 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [39]:
len(hf_tokenizer(raw_text, max_length=1024, truncation=True)["input_ids"])

1024